In [1]:
import sys, os, io
import pyodbc, pandas
import datetime

In [2]:
def writeInChunks(path, sql, cnxn, chunksize = 10000):
    """
    Writes an sql in chunk to avoid memory problems
    """ 
    with io.open(path, "w", encoding="utf-8") as fh:
        chunks = pandas.read_sql_query(sql, cnxn, chunksize=chunksize)
        next(chunks).to_csv(fh, index=False)  # write the first chunk with the column names,
                                              # but ignore the index (which will be screwed up anyway due to the chunking)
       
        for chunk in chunks:
            print('Chunk written')
            chunk.to_csv(fh, index=False, header=False) # skip the column names from now on


In [3]:
cnxn = pyodbc.connect(driver='{SQL Server}',
                      server= '192.168.27.210\CMRC',
                      database='SIOS1',
                      uid='CI',
                      pwd='123')
cur = cnxn.cursor()

In [4]:
sql = "SELECT * FROM Casos WHERE Paciente IN ( SELECT Id FROM Pacientes WHERE Identificacion LIKE 'VEN%')"


In [4]:
sqlVenezolanas = "SELECT Id FROM Pacientes WHERE Identificacion LIKE 'VEN%'"
def downloadPatientsAndRegistersFromPatients(cnxn, sqlPatients, pathWriteFolder):
    """
    Downloads  all the cases, registers and operations between two dates, plus/minus a certain slack to ensure all registers are fully recovered
    """
    try:
        os.mkdir(pathWriteFolder)
    except:
        pass
    sqlRegistros ="""SELECT * FROM RegistrosHistoria where NumeroHistoria in (%s)  """ % (sqlPatients)
    sqlCasos ="""SELECT * FROM Casos where Paciente in (%s) """ % (sqlPatients)
    sqlProcedimientos ="""SELECT * FROM ProgramacionQx.DescripcionQuirurgica where IdPaciente  in (%s)""" % (sqlPatients)
    sqlPacientes = """SELECT * FROM PACIENTES WHERE Id in (%s)""" %  sqlPatients
    
    sqlCasosID ="""SELECT Caso FROM Casos where Paciente in (%s) """ % (sqlPatients)
    sqlDatosEnfermeria = """SELECT * FROM Enfermeria.SignosVitalesControl WHERE IdAdmision IN (%s)""" % (sqlCasosID)

    
    writeInChunks(os.path.join(pathWriteFolder,'registros.csv'), sqlRegistros, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'casos.csv'), sqlCasos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'procedimientos.csv'), sqlProcedimientos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'pacientes.csv'), sqlPacientes, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'enfermeriaMedidas.csv'), sqlDatosEnfermeria, cnxn)
downloadPatientsAndRegistersFromPatients(cnxn, sqlVenezolanas, 'Venezolanas2')

Chunk written


In [14]:
def downloadPatientsAndRegistersFromDate(cnxn, startDate, endDate, pathWriteFolder,  slackDays = 15):
    """
    Downloads  all the cases, registers and operations between two dates, plus/minus a certain slack to ensure all registers are fully recovered
    """
    try:
        os.mkdir(pathWriteFolder)
    except:
        pass
    slackDaysDT = datetime.timedelta(days = slackDays)
    startDateDT = datetime.datetime.strptime(startDate, '%d/%m/%Y') + slackDaysDT
    endDateDT = datetime.datetime.strptime(endDate, '%d/%m/%Y') + slackDaysDT
    
    startDateStr =  startDateDT.strftime("%Y-%d-%m")
    endDateStr =endDateDT.strftime("%Y-%d-%m")
    sqlRegistros ="""SELECT * FROM RegistrosHistoria where FechaAsignacionRegistro < '%s' AND FechaAsignacionRegistro > '%s' """ % (endDateStr, startDateStr)
    sqlCasos ="""SELECT * FROM Casos where FechaHora < '%s' AND FechaHora > '%s' """ % (endDateStr, startDateStr)
    sqlProcedimientos ="""SELECT * FROM ProgramacionQx.DescripcionQuirurgica where FechaDescripcion < '%s' AND FechaDescripcion > '%s' """ % (endDateStr, startDateStr)
    sqlProcedimientosIDs = """SELECT DISTINCT IdPaciente FROM ProgramacionQx.DescripcionQuirurgica where FechaDescripcion < '%s' AND FechaDescripcion > '%s' """ % (endDateStr, startDateStr)
    sqlPacientes = """SELECT * FROM PACIENTES WHERE Id in (%s)""" %  sqlProcedimientosIDs
    
    sqlCasosID ="""SELECT Caso FROM Casos where FechaHora < '%s' AND FechaHora > '%s' """ % (endDateStr, startDateStr)
    sqlDatosEnfermeria = """SELECT * FROM Enfermeria.SignosVitalesControl WHERE IdAdmision IN (%s)""" % (sqlCasosID)

    
    writeInChunks(os.path.join(pathWriteFolder,'registros.csv'), sqlRegistros, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'casos.csv'), sqlCasos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'procedimientos.csv'), sqlProcedimientos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'pacientes.csv'), sqlPacientes, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'enfermeriaMedidas.csv'), sqlDatosEnfermeria, cnxn)


In [15]:
downloadPatientsAndRegistersFromDate(cnxn, '01/01/2019', '01/01/2020', 'casos2019')

Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written


In [101]:
cnxn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+ \
                      'DBQ=C:\\sip\\SIP_Respaldo\\sip20200302.mdb;')

In [186]:
df = pandas.read_csv('resultsSIP.csv', index_col = 0)
df['ID01'] = df['VAR_0019'].map(lambda s: '000000000008060061-80000000000' + s)
df['VAR_0198'] = df['VAR_0198'].map(lambda s: str(int(s)) if s == s else s)

In [187]:
fullCols = list(map(lambda s: s, cursor.columns(table='nivel_N1')))
colNames = list(map(lambda s: s[3], cursor.columns(table='nivel_N1')))
cols = []
values = []
i = 0
for c in colNames:
    
    if c in df and df.iloc[i][c] == df.iloc[i][c]:
        cols.append(c)
        values.append(df.iloc[i][c])

In [188]:
"insert into nivel_N1(%s) values (\'%s\')" % ( ','.join(cols), '\',\''.join(map(str,values)))

"insert into nivel_N1(ID01,VAR_0006,VAR_0012,VAR_0013,VAR_0015,VAR_0018,VAR_0019,VAR_0020,VAR_0022,VAR_0024,VAR_0026,VAR_0028,VAR_0030,VAR_0032,VAR_0040,VAR_0041,VAR_0046,VAR_0047,VAR_0055,VAR_0056,VAR_0057,VAR_0059,VAR_0183,VAR_0198,VAR_0282,VAR_0283,VAR_0284,VAR_0287,VAR_0293,VAR_0297,VAR_0298,VAR_0310,VAR_0311,VAR_0313,VAR_0314,VAR_0321,VAR_0322,VAR_0343,VAR_0425) values ('000000000008060061-80000000000VEN25322789','1996-03-28 00:00:00','B','C','D','CMRC','VEN25322789','A','A','B','A','A','A','A','2.0','0.0','2.0','0.0','65.0','60.0','07/06/1954','A','2016-08-27 22:59:00','39','A','02:25 a.m.','28/08/2016','A','X','B','A','B','2100.0','31.0','43.0','6.0','9.0','A','2016-08-30')"

In [184]:
v = "insert into nivel_N1(ID01,VAR_0006,VAR_0012,VAR_0013,VAR_0015,VAR_0018,VAR_0019,VAR_0020,VAR_0022,VAR_0024,VAR_0026,VAR_0028,VAR_0030,VAR_0032,VAR_0040,VAR_0041,VAR_0046,VAR_0047,VAR_0055,VAR_0056,VAR_0057,VAR_0059,VAR_0183) values ('000000000008060061-80000000000VEN25322789','1996-03-28 00:00:00','B','C','D','CMRC','VEN25322789','A','A','B','A','A','A','A','2.0','0.0','2.0','0.0','65.0','60.0','07/06/1954','A','2016-08-27 22:59:00')"

In [185]:
cursor.execute("delete from nivel_N1 where ID01='%s'" % values[0])
#cursor.execute("insert into nivel_N1(%s) values (\'%s\')" % ( ','.join(cols[:23]), '\',\''.join(map(str,values[:23]))))
cursor.execute

<function Cursor.execute>

In [28]:
cnxn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+ \
                      'DBQ=C:\\sip\\SIP_Respaldo\\sip20200302.mdb;')
cursor = cnxn.cursor()
for row in cursor.tables():
    print (row.table_name)


In [ ]:
cursor.execute("insert into nivel_N1(id, name) values ('pyodbc', 'awesome library')")
cnxn.commit()


In [5]:
diagnosis =  pandas.read_sql_query('SELECT * FROM Procedimientos ', cnxn)
diagnosis.to_csv('procedimientosID.csv', index=False)



In [4]:
diagnosis =  pandas.read_sql_query('SELECT * FROM Diagnosticos ', cnxn)
diagnosis.to_csv('diagnosis.csv', index=False) # skip the column names from now on


In [ ]:
diagnosis =  pandas.read_sql_query('SELECT * FROM Diagnosticos ', cnxn)
diagnosis.to_csv('diagnosis.csv', index=False) # skip the column names from now on


In [40]:
diagnosis.loc[diagnosis['CodDiag'] == 'Z370']

,CodDiag,Simbolo,Descrip,Sexo,LimInf,LimSup,NoAfeccion,Observacion,Clasificacion
11989,Z370,None,"NACIDO VIVO, UNICO",A,NaN,227.0,None,None,CIE10


In [37]:
import xml.dom.minidom

dom = xml.dom.minidom.parseString(registers.loc[8, 'RegistroXML']) # or xml.dom.minidom.parseString(xml_string)
pretty_xml_as_string = dom.toprettyxml()
print(pretty_xml_as_string)

<?xml version="1.0" ?>
<C Asunto="Evoluci&amp;#243;n" Caso="AD236491" CentroA="01" CodigoDiagnosticoRelacionado1="" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O623" IdPaciente="170478" Modulo="01" Padre="860231" PlanAdm="480110" Prestador="73102778" Raiz="94" Registro="106">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="SE TRASLADA A SALA DE PARTO #1 PARA REALIZACION DE AMNIOTOMIA.&amp;lt;BR&amp;gt;EN POSICION DE LITOTOMIA PREVIA ASEPSIA, ANTISEPSIA CON PRESENCIA DE FRECUENCIA CARDIACA FETAL PREAMNIOTOMIA DE 142 LPM SE REALIZA AMNIOTOMIA OBTENIENDO LIQUIDO CLARO, EUTERMICO, NO FETIDO, EN ABUNDANTE CANTIDAD, SE VERICA FRECUENCIA CARDIACA FETAL POSTERIOR A LA AMNIOTOMIA EN 144LPM. CONTINUAR CON VIGILANCIA DE TRABAJO DE PARTO."/>
	<row NombreCampo="FechaNota" NombreTabla="CamposFecha" ValorCampo="06/07/2017 13:37"/>
	<row NombreCampo="FechaEgreso" NombreTabla="CamposFecha" ValorCampo=""/>
	<row NombreCampo="Autorizacion" Nombr

In [26]:
for i,p in enumerate(registers['CodigoDiagnosticoRelacionado1']):
    print(i,  p)

0 None
1 Z014
2 Z014
3 None
4 
5 
6 
7 None
8 
9 None
10 Z370
11 None
12 None
13 
14 R104
15 None
16 Z488
17 D509


In [44]:
import pandas
DF = pandas.read_sql_query("SELECT TOP 5 * FROM RegistrosHistoria", cnxn)


In [45]:
DF

,Id,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
0,2625,68909,AD0004,07,Ingreso de Urgencias,2013-09-20 09:14:12.747,NULL,NULL,sa,73083700,...,False,None,None,None,03,None,None,None,False,None
1,2626,68909,TRI001,25,,2013-09-20 09:14:13.147,None,None,sa,73083700,...,False,None,None,None,None,None,None,None,False,None
2,2627,68909,AD0004,02,MOTIVACION PARA CESAREA,2013-09-20 10:04:44.483,NULL,NULL,sa,73083700,...,False,None,None,None,03,None,None,None,False,None
3,2628,68909,AD0004,14,EVOLUCION,2013-09-20 11:06:56.620,07,2625,sa,73083700,...,False,None,None,None,03,None,None,None,False,None
4,2629,68930,AD0012,02,SDEW,2013-09-20 16:52:21.880,NULL,NULL,sa,73080532,...,False,None,None,None,01,None,None,None,False,None


In [ ]:
columns = [column[0] for column in cur.description]

In [ ]:
columns

In [ ]:
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\sip\SIP_respaldo\sip20200207.mdb;')


In [ ]:
os.path.exists(MDB)

In [ ]:
cnxn = pyodbc.connect(driver='{SQL Server}', host=SRV, database=MDB,
                      trusted_connection=True)


In [ ]:
import subprocess
pathMDBTools = 'C:\\Users\\Public\\Gabriel\\mdbtools-win-master\\mdbtools-win-master'
DATABASE = 'C:\\sip\\SIP_respaldo\\sip20200207.mdb'
#!/usr/bin/env python
#
# AccessDump.py
# A simple script to dump the contents of a Microsoft Access Database.
# It depends upon the mdbtools suite:
#   http://sourceforge.net/projects/mdbtools/

import sys, subprocess # the subprocess module is new in python v 2.4

# Get the list of table names with "mdb-tables"
table_names = subprocess.Popen([os.path.join(pathMDBTools,"mdb-tables.exe"), "-1", DATABASE], 
                               stdout=subprocess.PIPE).communicate()[0]
tables = table_names.split(b'\n')

# Dump each table as a CSV file using "mdb-export",
# converting " " in table names to "_" for the CSV filenames.
for table in tables[1:]:
    if table != '':
        table = table.decode("utf-8").replace('\r', '')
        filename = table.replace(" ","_") + ".csv"
        file = open(filename.replace('\r', ''), 'w')
        print("Dumping " + table)
        contents = subprocess.Popen([os.path.join(pathMDBTools, "mdb-export"), DATABASE, table],
                                    stdout=subprocess.PIPE).communicate()[0]
        file.write(contents.decode("utf-8"))
        file.close()

In [ ]:
print(os.path.join(pathMDBTools,"mdb-tables.exe"), "-1", DATABASE)

In [ ]:
contents